In [1]:
import os
%cd /home/iir/work/ben/NCKU/IIR/SemEval2025_Task9
!pwd

/home/iir/work/ben/NCKU/IIR/SemEval2025_Task9
/home/iir/work/ben/NCKU/IIR/SemEval2025_Task9


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import datetime
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score
import torch.nn.functional as F
from src.MyDataset import MyDataset
from src.BertSentimentClassifier import BertSentimentClassifier
from src.utils import *
import warnings
import wandb
warnings.filterwarnings("ignore")

/root/anaconda3/envs/augllms/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_csv('data/incidents_train.csv', index_col=0)
project_name  = "ST1_Hazard_Category_V1"
wandb.init(
    project="Thesis_SemEval_2025_Task9",
    name=project_name, 
    config={
        "learning_rate": 2e-5,
        "epochs": 150,
        "batch_size": 400
})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: phanben110. Use `wandb login --relogin` to force relogin


In [4]:
name_bert = "alvaroalon2/biobert_diseases_ner"
tokenizer = BertTokenizer.from_pretrained(name_bert) 

In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd

def stratified_split(csv_path, stratify_column, test_size=0.2, random_state=2024, debug=False):
    """
    Split data into training and development sets using stratified splitting.
    
    Parameters:
    - csv_path (str): Path to the CSV file.
    - stratify_column (str): Column name to use for stratification.
    - test_size (float): Proportion of data to use for the development set (default = 0.2).
    - random_state (int): Seed for random number generator to ensure reproducibility (default = 2024).
    
    Returns:
    - train_df (pd.DataFrame): Training set.
    - dev_df (pd.DataFrame): Development set.
    """
    # Read data from CSV file
    data = pd.read_csv(csv_path, index_col=0)
    
    # Check if the stratify column exists
    if stratify_column not in data.columns:
        raise ValueError(f"Column '{stratify_column}' does not exist in the dataset")

    if debug: 
        # Check distribution of the classes
        print(f"Number of unique classes: {len(data[stratify_column].value_counts())}")
    
    # Perform stratified split
    train_df, dev_df = train_test_split(
        data, 
        test_size=test_size, 
        random_state=random_state, 
        stratify=data[stratify_column]
    )
    
    if debug:
        # Display class distribution in the split sets
        print("\nClass distribution in the training set:")
        print(train_df[stratify_column].value_counts())
        
        print("\nClass distribution in the development set:")
        print(dev_df[stratify_column].value_counts())
    
    return train_df, dev_df


In [6]:
train_df, val_df = stratified_split(csv_path="data/incidents_train.csv", stratify_column="hazard-category", debug=False)

In [7]:
label = "hazard-category"
label_encoder = LabelEncoder()
label_encoder.fit(data["hazard-category"])
train_df['label'] = label_encoder.transform(train_df[label])
val_df['label'] = label_encoder.transform(val_df[label])

In [12]:
train_df

,year,month,day,country,title,text,hazard-category,product-category,hazard,product,label
5530,2022,1,3,us,"GHGA, LLC. Issues Allergy Alert on Undeclared ...","January 3, 2022, Conley, GA – GHGA, LLC. is vo...",allergens,prepared dishes and snacks,fish and products thereof,sandwiches,0
1404,2015,10,22,au,Yuen's Market Trading Co—Chaokoh Young Coconut...,PRA No. 2015/14983 Date published 22 Oct 2015 ...,allergens,non-alcoholic beverages,milk and products thereof,coconut juice,0
3469,2019,5,3,ca,Feeding Change brand Young Thai Coconut Meat r...,Updated Food Recall Warning - Feeding Change b...,biological,fruits and vegetables,salmonella,coconuts,1
4589,2020,11,13,ie,Almond in coffee beans by Lorge from Ireland,Undeclared Whole Almonds in Lorge Roasted Coff...,allergens,"cocoa and cocoa preparations, coffee and tea",almond,coffee beans,0
2264,2017,6,17,us,Supreme Cuisine Recalls Meat and Poultry Fat a...,"WASHINGTON, June 17, 2017 – Supreme Cuisine, a...",other hazard,"meat, egg and dairy products",processing,bovine fat,8
...,...,...,...,...,...,...,...,...,...,...,...
2901,2018,7,1,us,Allergy Alert Issued in Midwest Whole Foods Ma...,Whole Foods Market is voluntarily recalling Wh...,allergens,cereals and bakery products,soybeans and products thereof,cookies,0
2969,2018,8,4,us,voluntary recall of hostess cookies 'n creme b...,"Kansas City, MO – Hostess Brands, LLC has beco...",allergens,cereals and bakery products,eggs and products thereof,cookies,0
672,2012,7,6,us,2012 - Pacific International Marketing Volunta...,"FOR IMMEDIATE RELEASE - July 6, 2012 - Pacific...",biological,fruits and vegetables,salmonella,lettuce,1
4957,2021,4,18,au,Destination Holdings Pty Ltd — Taylor Rd IGA P...,PRA No. 2021/18968 Date published 18 Apr 2021 ...,allergens,"meat, egg and dairy products",soybeans and products thereof,pork sausage,0


In [8]:
# Create data loaders
max_len = 50
batch_size = 400
num_epochs = 150  # Adjust as needed

train_dataset = MyDataset(train_df, tokenizer, max_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [9]:
val_dataset = MyDataset(val_df, tokenizer, max_len)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [10]:
# Define model and optimizer
num_classes = train_df["label"].nunique()
model = BertSentimentClassifier(name_bert, num_classes)
criterion = nn.CrossEntropyLoss()

Some weights of BertModel were not initialized from the model checkpoint at alvaroalon2/biobert_diseases_ner and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Create logs folder
logs_folder = "logs"
os.makedirs(logs_folder, exist_ok=True)

# Generate a filename for logging
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_filename = f"{logs_folder}/training_log_BERT1_{current_time}.txt"

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lr = 2e-5
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.to(device)

# Metrics lists
train_losses, train_precisions, train_recalls, train_f1_scores = [], [], [], []
val_losses, val_precisions, val_recalls, val_f1_scores = [], [], [], []

best_val_loss = float('inf')
best_epoch = 0

# Start training loop
with open(log_filename, 'a') as log_file:
    for epoch in range(num_epochs):
        start_time = datetime.now()
        print(f'Epoch {epoch + 1}/{num_epochs}')

        # Training
        train_loss, train_precision, train_recall, train_f1 = train(model, train_loader, optimizer, criterion, device)

        # Validation
        val_loss, val_precision, val_recall, val_f1 = evaluate(model, val_loader, criterion, device)


        # Log results
        wandb.log({
            'Train Loss': train_loss, 'Train Precision': train_precision, 'Train Recall': train_recall, 'Train F1': train_f1,
            'Val Loss': val_loss, 'Val Precision': val_precision, 'Val Recall': val_recall, 'Val F1': val_f1,
        })

        print(f'Train -- Loss: {train_loss:.4f} | Precision: {train_precision:.4f} | Recall: {train_recall:.4f} | F1: {train_f1:.4f}')
        print(f'Val   -- Loss: {val_loss:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}')

        log_file.write(f'Epoch {epoch + 1}/{num_epochs}\n')
        log_file.write(f'Train -- Loss: {train_loss:.4f} | Precision: {train_precision:.4f} | Recall: {train_recall:.4f} | F1: {train_f1:.4f}\n')
        log_file.write(f'Val   -- Loss: {val_loss:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}')
        log_file.flush()

        # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch + 1
            torch.save(model.state_dict(), f'best_model_BERT1.pt')

        # Save metrics
        train_losses.append(train_loss)
        train_precisions.append(train_precision)
        train_recalls.append(train_recall)
        train_f1_scores.append(train_f1)

        val_losses.append(val_loss)
        val_precisions.append(val_precision)
        val_recalls.append(val_recall)
        val_f1_scores.append(val_f1)

        # Calculate and print time remaining
        elapsed_time = datetime.now() - start_time
        remaining_time = (num_epochs - epoch - 1) * elapsed_time
        print(f'Time Elapsed: {elapsed_time}, Time Remaining: {remaining_time} \n')
        torch.save(model.state_dict(), f'final_model_BERT1.pt')

# Print the best epoch
print(f"Best Model: Epoch {best_epoch}, Best Validation Loss: {best_val_loss:.4f}")
print(f"Path log file: {log_filename}")

Epoch 1/150


100%|██████████| 3/3 [00:00<00:00,  3.45it/s]


Train -- Loss: 1.5421 | Precision: 0.2538 | Recall: 0.3244 | F1: 0.2415
Val   -- Loss: 1.7385 | Precision: 0.3956 | Recall: 0.3830 | F1: 0.2369
Time Elapsed: 0:00:05.626059, Time Remaining: 0:13:58.282791 

Epoch 2/150


100%|██████████| 3/3 [00:00<00:00,  3.45it/s]


Train -- Loss: 1.1407 | Precision: 0.3363 | Recall: 0.4638 | F1: 0.3793
Val   -- Loss: 1.5351 | Precision: 0.4882 | Recall: 0.5878 | F1: 0.5000
Time Elapsed: 0:00:05.139100, Time Remaining: 0:12:40.586800 

Epoch 3/150


...

100%|██████████| 3/3 [00:00<00:00,  4.88it/s]


Train -- Loss: 0.0041 | Precision: 0.9969 | Recall: 0.9964 | F1: 0.9964
Val   -- Loss: 0.7248 | Precision: 0.8305 | Recall: 0.8297 | F1: 0.8253
Time Elapsed: 0:00:03.881581, Time Remaining: 0:03:56.776441 

Epoch 90/150


100%|██████████| 3/3 [00:00<00:00,  4.88it/s]


Train -- Loss: 0.0051 | Precision: 0.9946 | Recall: 0.9938 | F1: 0.9939
Val   -- Loss: 0.7164 | Precision: 0.8278 | Recall: 0.8304 | F1: 0.8257
Time Elapsed: 0:00:03.872604, Time Remaining: 0:03:52.356240 

Epoch 91/150


 67%|██████▋   | 2/3 [00:00<00:00,  4.12it/s]